In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
%load_ext autoreload
%autoreload 2

# imports

In [ ]:
import os
import numpy as np
import pandas as pd
from natsort import natsorted
from scipy.stats import pearsonr 

In [ ]:
import decision_margin_consistency.analyses.self_consistency as analysis

# load the data

In [ ]:
exp_name = 'snr-edges-v1'
df = analysis.load_data(exp_name, nTrials=160)

In [ ]:
df.iloc[0]

In [ ]:
df.groupby(by=['workerID']).responseCorrect.mean()

# check for outliers

Trim any subjects more than 3 STD from the mean (there were none in this dataset)

In [ ]:
import seaborn as sns
subjects = df.workerID.unique()
accuracy = df.groupby(by=['workerID']).responseCorrect.mean()
M = accuracy.mean()
STD = accuracy.std()
lower = M - 3*STD 
upper = min(.99, M + 3*STD)
outliers = (accuracy < lower) | (accuracy > upper)
any(outliers)

In [ ]:
g = sns.displot(accuracy)
g.set(xlim=(.50, 1.00));

# self-consistency analysis

We computed self-consistency by comparing subjects to themselves (1st vs. 2nd trial across images). TLDR, subjects do not respond consistency across trials.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
results = analysis.compute_summary(df)
results

In [ ]:
condNames = sorted(results.condName.unique())
subjects = results.subject.unique()
condNames, subjects, len(subjects)

For each individual, compute the mean accuracy for the 1st and 2nd trial, and the correlation across items for trial1 accuracy vs. trial2 accuracy.

In [ ]:
trial1_acc = []
trial2_acc = []
corrs = []
r2s = []
N = len(subjects)
for subject in subjects:
    subset = results[results.subject==subject]
    corr = pearsonr(subset.correct1, subset.correct2)[0]
    r2 = corr**2
    corrs.append(corr)
    r2s.append(r2)
    trial1_acc.append(subset.correct1.mean())
    trial2_acc.append(subset.correct2.mean())
avg_trial1_acc = np.mean(trial1_acc)
avg_trial2_acc = np.mean(trial2_acc)
avg_corr = np.mean(corrs)
avg_r2 = np.mean(r2s)  
print(f"Summary of first vs. second response performance (N={N})")
print(f"Mean proportion correct first trial = {avg_trial1_acc:3.3f}, vs. second trial = {avg_trial2_acc:3.3f}")
print(f"Correlation across items first vs. second response, r={avg_corr:3.3f}, r2={avg_r2:3.3}")

Next we compute cohen's kappa (error consistency) between first and second response.

In [ ]:
print("\nConsistency of first vs. second response (within a subject) ==>")
kappas = []
corrs = []
for subject in subjects:
    subset = results[results.subject==subject]
    condName = subset.iloc[0].condName
    assert len(subset)==80
    err_con = analysis.compute_error_consistency(subset.correct1.values, subset.correct2.values)
    r = pearsonr(subset.correct1, subset.correct2)[0]
    kappas.append(err_con['k'])
    corrs.append(r)
    # print(f"{subject[0:5]}... ({condName}): c_exp={err_con['c_exp']:2.3f}, c_obs={err_con['c_obs']:2.3f} kappa={err_con['k']:2.3f}, r={r:2.3f}")

kappas = np.array(kappas)
corrs = np.array(corrs)
kappas.mean(), corrs.mean()

print(f"Cohen's kappa (average): {kappas.mean():3.3f}")
print(f"Accuracy correlation (average): {corrs.mean():3.3f}")

In [ ]:
len(kappas)

In [ ]:
ax = sns.scatterplot(x=kappas, y=corrs)
ax.axis('square');
ax.set_xlim([.3,1.0]);
ax.set_ylim([.3,1.0]);
ax.set_xlabel("Cohen's Kappa")
ax.set_ylabel("pearson r")
ax.plot([0, 1], [0, 1], transform=ax.transAxes)

In [ ]:
import pandas as pd
from collections import defaultdict

def get_group_avg_accuracy(df):
    results = defaultdict(list)
    for condName in condNames:
        df_ = df[df.condName==condName]
        all_items = natsorted(df_.item.unique())
        subjects = df_.subject.unique()

        for item in all_items:
            subset = df_[df_.item==item]
            assert len(subset)==len(subjects)
            results['condName'].append(condName)
            results['item'].append(item)
            results['correct1'].append(subset.correct1.mean())
            results['correct2'].append(subset.correct2.mean())

    results = pd.DataFrame(results)

    return results

avg_acc = get_group_avg_accuracy(results)
avg_acc

In [ ]:
for condName in condNames:
    subset = avg_acc[avg_acc.condName==condName]
    assert len(subset)==80  
    r = pearsonr(subset.correct1, subset.correct2)[0]
    print(f"{condName}: r={r:3.3f}, r\u00B2={r*r:3.3f}")

In [ ]:
def fisherz(r, eps=1e-5):
    return np.arctanh(r-eps)

def fisherz_inv(z):
    return np.tanh(z)

In [ ]:
import matplotlib.pyplot as plt 

for condName in condNames:
    subset = avg_acc[avg_acc.condName==condName]
    assert len(subset)==80  
    r = pearsonr(subset.correct1, subset.correct2)[0]
    print(f"{condName}: r={r:3.3f}")

    ax = sns.scatterplot(x=subset.correct1, y=subset.correct2)
    ax.axis('square');
    ax.set_xlim([0,1.2]);
    ax.set_ylim([0,1.2]);
    ax.set_xlabel("accuracy first presentation")
    ax.set_ylabel("accuracy second presentation")
    plt.show()

In [ ]:
corrs = []
corrs_half = []
corrs_12 = []
corrs_21 = []
r1s = []
r2s = []
for condName in condNames:
    df_ = results[results.condName==condName]
    all_items = df_.item.unique()
    subjects = df_.subject.unique()
    num_subj = len(subjects)
    for idx1 in range(0,num_subj-1):
        sub1 = subjects[idx1]
        dat1 = df_[df_.subject==sub1].reset_index()
        assert len(dat1)==80
        for idx2 in range(idx1+1,num_subj):
            sub2 = subjects[idx2]      
            dat2 = df_[df_.subject==sub2].reset_index()
            assert len(dat1)==80
            assert all((dat1.subject == dat2.subject)==False)
            assert all((dat1.item == dat2.item)==True)
            r = pearsonr(dat1.correctAvg, dat2.correctAvg)[0]
            r1 = pearsonr(dat1.correct1, dat1.correct2)[0]
            r2 = pearsonr(dat2.correct1, dat2.correct2)[0]
            r12 = pearsonr(dat1.correct1, dat2.correct2)[0]
            r21 = pearsonr(dat1.correct2, dat2.correct1)[0]
            rhalf = (r12+r21)/2
            corrs.append(r)
            corrs_half.append(rhalf)
            corrs_12.append(r12)
            corrs_21.append(r21)
            r1s.append(r1)
            r2s.append(r2)
            print(f"{condName}-Sub{idx1}-Sub{idx2}: r={r:3.3f}, rhalf={rhalf:3.3f}, r12={r12:3.3f}, r21={r21:3.3f}")

corrs = np.array(corrs)
corrs_half = np.array(corrs_half)
corrs_12 = np.array(corrs_half)
corrs_21 = np.array(corrs_21)
r1s = np.array(r1s)
r2s = np.array(r2s)
corrs.mean(), corrs_half.mean(), r1s.mean(), r2s.mean()

In [ ]:
2*corrs_half.mean() / (1+corrs_half.mean())

In [ ]:
rAB = corrs_half.mean() / np.sqrt(r1s.mean() * r2s.mean())
rAB

In [ ]:
2*rAB / (1+rAB)